## Logistic Regression मोडेल बनाउनुहोस् - पाठ ४

![Logistic vs. linear regression infographic](../../../../../../translated_images/linear-vs-logistic.ba180bf95e7ee66721ba10ebf2dac2666acbd64a88b003c83928712433a13c7d.ne.png)

#### **[पाठ अघि क्विज](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### परिचय

Regression को यो अन्तिम पाठमा, जुन *classic* ML प्रविधिहरू मध्ये एक हो, हामी Logistic Regression को अध्ययन गर्नेछौं। तपाईंले यो प्रविधि प्रयोग गरेर ढाँचाहरू पत्ता लगाउन सक्नुहुन्छ जसले द्विआधारी श्रेणीहरूको भविष्यवाणी गर्न मद्दत गर्दछ। यो क्यान्डी चकलेट हो कि होइन? यो रोग संक्रामक हो कि होइन? यो ग्राहकले यो उत्पादन रोज्नेछ कि छैन?

यस पाठमा, तपाईंले सिक्नुहुनेछ:

-   Logistic Regression का प्रविधिहरू

✅ यो प्रकारको Regression मा काम गर्ने आफ्नो बुझाइलाई यो [Learn module](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott) मा गहिरो बनाउनुहोस्।

## पूर्वशर्त

Pumpkin डेटा संग काम गरेपछि, हामी यससँग पर्याप्त परिचित छौं कि हामीले महसुस गर्न सक्छौं कि त्यहाँ एउटा द्विआधारी श्रेणी छ जसमा हामी काम गर्न सक्छौं: `Color`।

आउनुहोस्, Logistic Regression मोडेल बनाऔं जसले केही चरहरू दिइएको अवस्थामा *कुन रंगको Pumpkin हुने सम्भावना छ* (orange 🎃 वा white 👻) भनेर भविष्यवाणी गर्न सक्छ।

> किन हामी Regression को पाठ समूहमा द्विआधारी वर्गीकरणको कुरा गर्दैछौं? केवल भाषिक सुविधाको लागि, किनकि Logistic Regression [वास्तवमा एक वर्गीकरण विधि हो](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), यद्यपि यो linear-आधारित हो। अर्को पाठ समूहमा डेटा वर्गीकरण गर्ने अन्य तरिकाहरूको बारेमा जान्नुहोस्।

यस पाठको लागि, हामीलाई निम्न प्याकेजहरू आवश्यक पर्नेछ:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) [R प्याकेजहरूको संग्रह](https://www.tidyverse.org/packages) हो जसले डेटा विज्ञानलाई छिटो, सजिलो र रमाइलो बनाउँछ!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) फ्रेमवर्क [प्याकेजहरूको संग्रह](https://www.tidymodels.org/packages/) हो जसले मोडेलिङ र मेसिन लर्निङको लागि काम गर्दछ।

-   `janitor`: [janitor प्याकेज](https://github.com/sfirke/janitor) ले फोहोर डेटा जाँच र सफा गर्न साना उपकरणहरू प्रदान गर्दछ।

-   `ggbeeswarm`: [ggbeeswarm प्याकेज](https://github.com/eclarke/ggbeeswarm) ले ggplot2 प्रयोग गरेर beeswarm-शैलीका प्लटहरू बनाउनका लागि विधिहरू प्रदान गर्दछ।

तपाईंले यी प्याकेजहरू यसरी इन्स्टल गर्न सक्नुहुन्छ:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

वैकल्पिक रूपमा, तलको स्क्रिप्टले जाँच गर्दछ कि तपाईंले यो मोड्युल पूरा गर्न आवश्यक प्याकेजहरू छ कि छैन, र यदि छैन भने, तिनीहरूलाई तपाईंको लागि इन्स्टल गर्दछ।


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **प्रश्न परिभाषित गर्नुहोस्**

हाम्रो उद्देश्यका लागि, हामी यसलाई दुईवटा श्रेणीमा व्यक्त गर्नेछौं: 'सेतो' वा 'सेतो होइन'। हाम्रो डेटासेटमा 'धारीदार' नामको अर्को श्रेणी पनि छ, तर यसको उदाहरणहरू कम छन्, त्यसैले हामी यसलाई प्रयोग गर्नेछैनौं। यो श्रेणी डेटासेटबाट null मानहरू हटाएपछि हराउँछ।

> 🎃 रमाइलो तथ्य, हामी कहिलेकाहीं सेतो कद्दूलाई 'भूत' कद्दू भन्छौं। तिनीहरू कुँद्न सजिलो हुँदैनन्, त्यसैले तिनीहरू सुन्तला रंगका कद्दूहरू जत्तिकै लोकप्रिय छैनन्, तर तिनीहरू आकर्षक देखिन्छन्! त्यसैले हामी हाम्रो प्रश्नलाई यसरी पनि पुनःव्यक्त गर्न सक्छौं: 'भूत' वा 'भूत होइन'। 👻

## **लजिस्टिक रिग्रेसनको बारेमा**

लजिस्टिक रिग्रेसनले केही महत्त्वपूर्ण तरिकामा लीनियर रिग्रेसनबाट फरक छ, जुन तपाईंले पहिले सिक्नुभएको थियो।

#### **द्विविधात्मक वर्गीकरण**

लजिस्टिक रिग्रेसनले लीनियर रिग्रेसनजस्तै विशेषताहरू प्रदान गर्दैन। लजिस्टिक रिग्रेसनले `द्विविधात्मक श्रेणी` ("सुन्तला वा सुन्तला होइन") को भविष्यवाणी प्रदान गर्छ भने लीनियर रिग्रेसनले `निरन्तर मानहरू`को भविष्यवाणी गर्न सक्षम छ, जस्तै कद्दूको उत्पत्ति र कटनीको समय दिइएको अवस्थामा, *यसको मूल्य कति बढ्नेछ*।

![दासानी मडिपल्लीद्वारा इन्फोग्राफिक](../../../../../../translated_images/pumpkin-classifier.562771f104ad5436b87d1c67bca02a42a17841133556559325c0a0e348e5b774.ne.png)

### अन्य वर्गीकरणहरू

लजिस्टिक रिग्रेसनका अन्य प्रकारहरू पनि छन्, जस्तै मल्टिनोमियल र अर्डिनल:

- **मल्टिनोमियल**, जसमा एकभन्दा बढी श्रेणीहरू हुन्छन् - "सुन्तला, सेतो, र धारीदार"।

- **अर्डिनल**, जसमा क्रमबद्ध श्रेणीहरू हुन्छन्, यदि हामी हाम्रो परिणामहरूलाई तार्किक रूपमा क्रमबद्ध गर्न चाहन्छौं भने उपयोगी हुन्छ, जस्तै कद्दूहरू जुन निश्चित संख्याका आकारहरू (mini, sm, med, lg, xl, xxl) द्वारा क्रमबद्ध गरिएका छन्।

![मल्टिनोमियल बनाम अर्डिनल रिग्रेसन](../../../../../../translated_images/multinomial-vs-ordinal.36701b4850e37d86c9dd49f7bef93a2f94dbdb8fe03443eb68f0542f97f28f29.ne.png)

#### **भेरिएबलहरू अनिवार्य रूपमा सम्बन्धित हुनु पर्दैन**

तपाईंलाई याद छ लीनियर रिग्रेसनले बढी सम्बन्धित भेरिएबलहरूसँग राम्रो काम गर्थ्यो? लजिस्टिक रिग्रेसन यसको विपरीत हो - भेरिएबलहरू अनिवार्य रूपमा मिल्नुपर्दैन। यो डेटा, जसको सम्बन्धहरू केही कमजोर छन्,का लागि उपयुक्त छ।

#### **तपाईंलाई धेरै सफा डेटा चाहिन्छ**

लजिस्टिक रिग्रेसनले बढी डेटा प्रयोग गर्दा बढी सटीक परिणाम दिन्छ; हाम्रो सानो डेटासेट यो कार्यका लागि उपयुक्त छैन, त्यसैले यो कुरा ध्यानमा राख्नुहोस्।

✅ सोच्नुहोस् कि कुन प्रकारको डेटा लजिस्टिक रिग्रेसनका लागि उपयुक्त हुन्छ।

## अभ्यास - डेटा सफा गर्नुहोस्

पहिले, डेटा अलि सफा गर्नुहोस्, null मानहरू हटाएर र केही स्तम्भहरू मात्र चयन गरेर:

1. निम्न कोड थप्नुहोस्:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


तपाईंले आफ्नो नयाँ डेटा फ्रेमलाई सधैं हेर्न सक्नुहुन्छ, तल दिइएको [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) फंक्शन प्रयोग गरेर:


In [ ]:
pumpkins_select %>% 
  glimpse()


हामी वास्तवमा द्विआधारी वर्गीकरण समस्या गर्नेछौं भनेर पुष्टि गरौं:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### दृश्यात्मकता - श्रेणीगत प्लट
अहिलेसम्म तपाईंले फेरी पनि कद्दूको डेटा लोड गरेर सफा गर्नुभएको छ जसले केही भेरिएबलहरू सहितको डेटासेट सुरक्षित गर्दछ, जस्तै रंग। अब ggplot लाइब्रेरी प्रयोग गरेर नोटबुकमा डेटा फ्रेमलाई दृश्यात्मक बनाऔं।

ggplot लाइब्रेरीले तपाईंको डेटा दृश्यात्मक बनाउन केही राम्रो तरिकाहरू प्रदान गर्दछ। उदाहरणका लागि, तपाईं प्रत्येक प्रकार र रंगको लागि डेटा वितरणलाई श्रेणीगत प्लटमा तुलना गर्न सक्नुहुन्छ।

1. geombar फङ्सन प्रयोग गरेर यस्तो प्लट बनाउनुहोस्, हाम्रो कद्दूको डेटा प्रयोग गर्दै, र प्रत्येक कद्दूको श्रेणी (नारंगी वा सेतो) को लागि रंग म्यापिङ निर्दिष्ट गर्दै:


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

दिइएको डेटा हेर्दा, तपाईं देख्न सक्नुहुन्छ कि रंग डेटा किसिमसँग कसरी सम्बन्धित छ।

✅ यो श्रेणीगत प्लटलाई हेर्दा, तपाईंले के के रोचक अन्वेषणहरू कल्पना गर्न सक्नुहुन्छ?


### डाटा पूर्व-प्रक्रिया: विशेषता एन्कोडिङ

हाम्रो कद्दूको डेटासेटमा सबै स्तम्भहरूमा स्ट्रिङ मानहरू छन्। श्रेणीगत डाटासँग काम गर्नु मानिसहरूका लागि सहज हुन्छ तर मेसिनका लागि होइन। मेसिन लर्निङ एल्गोरिदमहरू संख्याहरूको साथ राम्रोसँग काम गर्छन्। त्यसैले एन्कोडिङ डाटा पूर्व-प्रक्रिया चरणमा एकदम महत्त्वपूर्ण कदम हो, किनभने यसले श्रेणीगत डाटालाई संख्यात्मक डाटामा रूपान्तरण गर्न सक्षम बनाउँछ, कुनै पनि जानकारी गुमाउनु बिना। राम्रो एन्कोडिङले राम्रो मोडेल निर्माण गर्न मद्दत गर्दछ।

विशेषता एन्कोडिङका लागि दुई मुख्य प्रकारका एन्कोडरहरू छन्:

1. **ओर्डिनल एन्कोडर**: यो ओर्डिनल भेरिएबलहरूका लागि उपयुक्त हुन्छ, जुन श्रेणीगत भेरिएबलहरू हुन् जहाँ तिनीहरूको डाटा तार्किक क्रमअनुसार हुन्छ, जस्तै हाम्रो डेटासेटको `item_size` स्तम्भ। यसले एउटा म्यापिङ सिर्जना गर्छ जसले प्रत्येक श्रेणीलाई एउटा संख्याले प्रतिनिधित्व गर्छ, जुन स्तम्भमा श्रेणीको क्रम हो।

2. **श्रेणीगत एन्कोडर**: यो नोमिनल भेरिएबलहरूका लागि उपयुक्त हुन्छ, जुन श्रेणीगत भेरिएबलहरू हुन् जहाँ तिनीहरूको डाटा तार्किक क्रमअनुसार हुँदैन, जस्तै हाम्रो डेटासेटमा `item_size` बाहेकका सबै विशेषताहरू। यो एक वन-हट एन्कोडिङ हो, जसको अर्थ प्रत्येक श्रेणीलाई एउटा बाइनरी स्तम्भले प्रतिनिधित्व गर्छ: एन्कोड गरिएको भेरिएबल 1 बराबर हुन्छ यदि कद्दू सो प्रकारको हो भने, र अन्यथा 0।

Tidymodels ले अर्को उपयोगी प्याकेज प्रदान गर्दछ: [recipes](https://recipes.tidymodels.org/) - डाटा पूर्व-प्रक्रियाको लागि एक प्याकेज। हामी एउटा `recipe` परिभाषित गर्नेछौं जसले निर्दिष्ट गर्छ कि सबै भविष्यवक्ता स्तम्भहरूलाई पूर्णांकहरूको सेटमा एन्कोड गर्नुपर्छ, `prep` गरेर आवश्यक मात्राहरू र कुनै पनि अपरेशनहरूका लागि आवश्यक तथ्याङ्कहरू अनुमान गर्नेछ, र अन्तमा `bake` गरेर नयाँ डाटामा गणनाहरू लागू गर्नेछ।

> सामान्यतया, recipes प्रायः मोडेलिङको लागि पूर्व-प्रक्रियाको रूपमा प्रयोग गरिन्छ जहाँ यसले डाटासेटलाई मोडेलिङको लागि तयार पार्न के कदमहरू लागू गर्नुपर्छ भनेर परिभाषित गर्छ। यस्तो अवस्थामा, **अत्यधिक सिफारिस गरिन्छ** कि तपाईं `workflow()` प्रयोग गर्नुहोस्, `prep` र `bake` प्रयोग गरेर म्यानुअल रूपमा recipe अनुमान नगरी। हामी यो सबै केही समयपछि देख्नेछौं।
>
> तर अहिलेका लागि, हामी recipes + prep + bake प्रयोग गर्दैछौं जसले डाटासेटमा लागू गर्नुपर्ने कदमहरू निर्दिष्ट गर्छ डाटा विश्लेषणको लागि तयार पार्न र त्यसपछि लागू गरिएका कदमहरू सहित पूर्व-प्रक्रिया गरिएको डाटा निकाल्न।


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ वस्तु आकार स्तम्भको लागि ओर्डिनल एनकोडर प्रयोग गर्दा के फाइदाहरू छन्?

### भेरिएबलहरू बीचको सम्बन्धको विश्लेषण गर्नुहोस्

अब हामीले हाम्रो डेटा पूर्व-प्रक्रिया गरिसकेपछि, हामी विशेषताहरू र लेबल बीचको सम्बन्धको विश्लेषण गर्न सक्छौं ताकि विशेषताहरू दिँदा मोडेलले लेबललाई कत्तिको राम्रोसँग भविष्यवाणी गर्न सक्छ भन्ने विचार प्राप्त गर्न सकियोस्। यस प्रकारको विश्लेषण गर्नको लागि सबैभन्दा राम्रो तरिका भनेको डेटा प्लट गर्नु हो।  
हामी फेरि ggplot को geom_boxplot_ फङ्सन प्रयोग गर्नेछौं, वस्तु आकार, भेराइटी, र रंग बीचको सम्बन्धलाई श्रेणीगत प्लटमा देखाउन। डेटा राम्रोसँग प्लट गर्नको लागि हामी एनकोड गरिएको वस्तु आकार स्तम्भ र एनकोड नगरिएको भेराइटी स्तम्भ प्रयोग गर्नेछौं।


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### स्वार्म प्लट प्रयोग गर्नुहोस्

किनभने रंग एक द्विआधारी श्रेणी (सेतो वा होइन) हो, यसलाई दृश्यात्मकता गर्न 'एक [विशेष विधि](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)' आवश्यक छ।

रंगको वितरणलाई वस्तुको आकारसँग सम्बन्धित देखाउन `स्वार्म प्लट` प्रयोग गर्नुहोस्।

हामी [ggbeeswarm प्याकेज](https://github.com/eclarke/ggbeeswarm) प्रयोग गर्नेछौं, जसले ggplot2 प्रयोग गरेर बीस्वार्म-शैलीका प्लटहरू बनाउनका लागि विधिहरू प्रदान गर्दछ। बीस्वार्म प्लटहरू ती बिन्दुहरूलाई प्लट गर्ने तरिका हो जुन सामान्यतया एकअर्कासँग ओभरलैप हुनेछन्, ताकि तिनीहरू एकअर्काको छेउमा पर्न सकून्।


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


अब हामीले रङका दुईवटा श्रेणीहरू र आकारहरूको ठूलो समूहको सम्बन्धको बारेमा विचार गरेका छौं, अब हामी एक कद्दूको सम्भावित रङ निर्धारण गर्नको लागि लजिस्टिक रिग्रेसनको अन्वेषण गरौं।

## आफ्नो मोडेल निर्माण गर्नुहोस्

तपाईं आफ्नो वर्गीकरण मोडेलमा प्रयोग गर्न चाहिने भेरिएबलहरू चयन गर्नुहोस् र डाटालाई प्रशिक्षण र परीक्षण सेटहरूमा विभाजन गर्नुहोस्। [rsample](https://rsample.tidymodels.org/), Tidymodels मा रहेको एउटा प्याकेजले डाटा विभाजन र पुनः नमूना गर्नको लागि प्रभावकारी संरचना प्रदान गर्दछ:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 अब हामी मोडेललाई प्रशिक्षण दिन तयार छौं, जहाँ प्रशिक्षण विशेषताहरूलाई प्रशिक्षण लेबल (रंग) मा फिट गरिन्छ।

हामीले हाम्रो डाटालाई मोडेलिङको लागि तयार पार्न आवश्यक पूर्वप्रक्रिया चरणहरू निर्दिष्ट गर्ने एउटा रेसिपी सिर्जना गरेर सुरु गर्नेछौं। उदाहरणका लागि: श्रेणीगत भेरिएबलहरूलाई पूर्णांकमा एन्कोड गर्नु। ठीक `baked_pumpkins` जस्तै, हामी `pumpkins_recipe` सिर्जना गर्छौं तर `prep` र `bake` गर्दैनौं किनभने यो वर्कफ्लोमा समेटिनेछ, जुन तपाईं केही चरणहरू पछि देख्नुहुनेछ।

Tidymodels मा logistic regression मोडेल निर्दिष्ट गर्ने धेरै तरिकाहरू छन्। `?logistic_reg()` हेर्नुहोस्। अहिलेको लागि, हामी डिफल्ट `stats::glm()` इन्जिन मार्फत logistic regression मोडेल निर्दिष्ट गर्नेछौं।


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


अब हामीसँग एउटा रेसिपी र मोडलको स्पेसिफिकेसन छ, हामीले यी दुवैलाई एकसाथ बाँधेर एउटा वस्तुमा राख्ने तरिका खोज्नुपर्छ, जसले पहिलो चरणमा डाटालाई पूर्वप्रक्रिया (भित्री रूपमा prep+bake) गर्नेछ, त्यसपछि पूर्वप्रक्रिया गरिएको डाटामा मोडललाई फिट गर्नेछ, र सम्भावित पोस्ट-प्रोसेसिङ गतिविधिहरूको लागि पनि अनुमति दिनेछ।

Tidymodels मा, यो सुविधाजनक वस्तुलाई [`workflow`](https://workflows.tidymodels.org/) भनिन्छ, जसले तपाईका मोडलिङ कम्पोनेन्टहरूलाई सजिलैसँग समेट्छ।


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


एक पटक वर्कफ्लो *निर्दिष्ट* भएपछि, मोडेललाई [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) फङ्सन प्रयोग गरेर `प्रशिक्षित` गर्न सकिन्छ। वर्कफ्लोले रेसिपीको अनुमान लगाउनेछ र प्रशिक्षण अघि डेटा पूर्वप्रक्रिया गर्नेछ, त्यसैले हामीलाई म्यानुअली prep र bake प्रयोग गरेर गर्न आवश्यक पर्दैन।


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


मोडेलले प्रशिक्षणको क्रममा सिकेका गुणांकहरू देखाउँछ।

अब हामीले प्रशिक्षण डाटाको प्रयोग गरेर मोडेललाई प्रशिक्षण गरिसकेपछि, हामी [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) प्रयोग गरेर परीक्षण डाटामा भविष्यवाणी गर्न सक्छौं। सुरुमा, मोडेल प्रयोग गरेर हाम्रो परीक्षण सेटका लागि लेबलहरू र प्रत्येक लेबलको सम्भावनाहरू भविष्यवाणी गरौं। जब सम्भावना ०.५ भन्दा बढी हुन्छ, भविष्यवाणी गरिएको वर्ग `WHITE` हुन्छ, अन्यथा `ORANGE`।


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


यो धेरै राम्रो छ! यसले logistic regression कसरी काम गर्छ भन्ने बारेमा थप जानकारी प्रदान गर्दछ।

### भ्रम निवारण म्याट्रिक्स मार्फत राम्रो बुझाइ

प्रत्येक भविष्यवाणीलाई यसको "ग्राउन्ड ट्रुथ" वास्तविक मानसँग तुलना गर्नु भनेको मोडेलले कत्तिको राम्रो भविष्यवाणी गरिरहेको छ भन्ने निर्धारण गर्ने कुशल तरिका होइन। भाग्यवश, Tidymodelsसँग केही थप उपायहरू छन्: [`yardstick`](https://yardstick.tidymodels.org/) - प्रदर्शन मेट्रिक्स प्रयोग गरेर मोडेलहरूको प्रभावकारिता मापन गर्न प्रयोग गरिने एक प्याकेज।

वर्गीकरण समस्यासँग सम्बन्धित एक प्रदर्शन मेट्रिक्स भनेको [`confusion matrix`](https://wikipedia.org/wiki/Confusion_matrix) हो। एक confusion matrix ले वर्गीकरण मोडेलले कत्तिको राम्रो प्रदर्शन गरिरहेको छ भन्ने वर्णन गर्दछ। confusion matrix ले प्रत्येक वर्गमा कति उदाहरणहरू सही रूपमा वर्गीकृत गरिएका छन् भन्ने ट्याबुलेट गर्दछ। हाम्रो केसमा, यसले देखाउनेछ कि कति सुन्तला रंगका कद्दूहरू सुन्तला रंगका रूपमा वर्गीकृत गरिए र कति सेतो कद्दूहरू सेतो रूपमा वर्गीकृत गरिए; confusion matrix ले यो पनि देखाउनेछ कि कति गलत वर्गहरूमा वर्गीकृत गरिए।

[`conf_mat()`](https://tidymodels.github.io/yardstick/reference/conf_mat.html) नामक yardstick को फङ्सनले अवलोकन गरिएका र भविष्यवाणी गरिएका वर्गहरूको क्रस-ट्याबुलेशन गणना गर्दछ।


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


आउनुहोस् भ्रमित म्याट्रिक्सलाई व्याख्या गरौं। हाम्रो मोडेललाई दुई बाइनरी श्रेणीहरू, श्रेणी `सेतो` र श्रेणी `सेतो नभएको` बीचमा कद्दू वर्गीकरण गर्न भनिएको छ।

-   यदि तपाईंको मोडेलले कद्दूलाई सेतो भनेर भविष्यवाणी गर्छ र वास्तवमा त्यो श्रेणी 'सेतो' मा पर्छ भने हामी यसलाई `साँचो सकारात्मक` भन्छौं, जुन माथिल्लो बायाँ नम्बरले देखाउँछ।

-   यदि तपाईंको मोडेलले कद्दूलाई सेतो नभएको भनेर भविष्यवाणी गर्छ र वास्तवमा त्यो श्रेणी 'सेतो' मा पर्छ भने हामी यसलाई `झुटो नकारात्मक` भन्छौं, जुन तल्लो बायाँ नम्बरले देखाउँछ।

-   यदि तपाईंको मोडेलले कद्दूलाई सेतो भनेर भविष्यवाणी गर्छ र वास्तवमा त्यो श्रेणी 'सेतो नभएको' मा पर्छ भने हामी यसलाई `झुटो सकारात्मक` भन्छौं, जुन माथिल्लो दायाँ नम्बरले देखाउँछ।

-   यदि तपाईंको मोडेलले कद्दूलाई सेतो नभएको भनेर भविष्यवाणी गर्छ र वास्तवमा त्यो श्रेणी 'सेतो नभएको' मा पर्छ भने हामी यसलाई `साँचो नकारात्मक` भन्छौं, जुन तल्लो दायाँ नम्बरले देखाउँछ।

| सत्यता |
|:-----:|


|               |        |       |
|---------------|--------|-------|
| **भविष्यवाणी गरिएको** | सेतो | सुन्तला |
| सेतो        | TP     | FP    |
| सुन्तला         | FN     | TN    |

जसरी तपाईंले अनुमान गर्नुभयो, साँचो सकारात्मक र साँचो नकारात्मकको संख्या धेरै हुनु राम्रो हो, र झुटो सकारात्मक र झुटो नकारात्मकको संख्या कम हुनु राम्रो हो, जसले मोडेल राम्रो प्रदर्शन गरेको संकेत गर्छ।

भ्रमित म्याट्रिक्स उपयोगी छ किनभने यसले अन्य मेट्रिक्सलाई जन्म दिन्छ जसले वर्गीकरण मोडेलको प्रदर्शनलाई राम्रोसँग मूल्याङ्कन गर्न मद्दत गर्न सक्छ। आउनुहोस् ती मध्ये केहीलाई हेरौं:

🎓 Precision: `TP/(TP + FP)` भविष्यवाणी गरिएको सकारात्मकहरू मध्ये वास्तवमा सकारात्मकको अनुपातको रूपमा परिभाषित गरिएको। यसलाई [सकारात्मक भविष्यवाणी मूल्य](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value") पनि भनिन्छ।

🎓 Recall: `TP/(TP + FN)` वास्तवमा सकारात्मक नमूनाहरूको संख्याबाट सकारात्मक परिणामहरूको अनुपातको रूपमा परिभाषित गरिएको। यसलाई `संवेदनशीलता` पनि भनिन्छ।

🎓 Specificity: `TN/(TN + FP)` वास्तवमा नकारात्मक नमूनाहरूको संख्याबाट नकारात्मक परिणामहरूको अनुपातको रूपमा परिभाषित गरिएको।

🎓 Accuracy: `TP + TN/(TP + TN + FP + FN)` नमूनाको लागि सही रूपमा भविष्यवाणी गरिएको लेबलहरूको प्रतिशत।

🎓 F Measure: Precision र Recall को भारित औसत, जसको सबैभन्दा राम्रो मान 1 हुन्छ र सबैभन्दा खराब मान 0 हुन्छ।

आउनुहोस् यी मेट्रिक्स गणना गरौं!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## यस मोडेलको ROC वक्रलाई दृश्यात्मक बनाउनुहोस्

अब हामी अर्को दृश्यात्मकता गर्नेछौं ताकि तथाकथित [`ROC वक्र`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) हेर्न सकियोस्:


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC वक्रहरू प्रायः वर्गीकरणकर्ताको नतिजालाई यसको सही र गलत सकारात्मकहरूको सन्दर्भमा हेर्न प्रयोग गरिन्छ। ROC वक्रहरू सामान्यतया Y अक्षमा `True Positive Rate`/संवेदनशीलता र X अक्षमा `False Positive Rate`/1-विशिष्टता देखाउँछन्। त्यसैले, वक्रको तीव्रता र मध्यरेखा र वक्रको बीचको स्थान महत्त्वपूर्ण हुन्छ: तपाईं चाहनुहुन्छ कि वक्र छिट्टै माथि र रेखा पार गरेर जाओस्। हाम्रो अवस्थामा, सुरुमा केही गलत सकारात्मकहरू छन्, त्यसपछि रेखा ठीकसँग माथि र पार जान्छ।

अन्तमा, `yardstick::roc_auc()` प्रयोग गरेर वक्रको क्षेत्रफल (Area Under the Curve) गणना गरौं। AUC व्याख्या गर्ने एउटा तरिका भनेको यो हो कि मोडेलले कुनै पनि जस्तोसुकै सकारात्मक उदाहरणलाई कुनै पनि जस्तोसुकै नकारात्मक उदाहरणभन्दा उच्च स्थानमा राख्ने सम्भावना हो।


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


नतिजा करिब `0.975` छ। AUC ० देखि १ सम्मको दायरामा हुन्छ, र तपाईं ठूलो स्कोर चाहनुहुन्छ, किनभने यदि कुनै मोडेलले आफ्ना भविष्यवाणीहरूमा १००% सहि छ भने, त्यसको AUC १ हुनेछ। यस अवस्थामा, मोडेल *धेरै राम्रो* छ।

भविष्यका वर्गीकरण सम्बन्धी पाठहरूमा, तपाईंले आफ्नो मोडेलको स्कोर सुधार गर्ने तरिकाहरू सिक्नुहुनेछ (जस्तै, यस अवस्थामा असन्तुलित डाटासँग व्यवहार गर्ने)।

## 🚀चुनौती

लजिस्टिक रिग्रेसनको बारेमा अझ धेरै कुरा जान्न बाँकी छ! तर सिक्ने सबैभन्दा राम्रो तरिका भनेको प्रयोग गर्नु हो। यस्तो डाटासेट खोज्नुहोस् जुन यस प्रकारको विश्लेषणका लागि उपयुक्त हो र त्यसको लागि मोडेल निर्माण गर्नुहोस्। तपाईंले के सिक्नुहुन्छ? सुझाव: [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) मा रोचक डाटासेटहरू खोज्नुहोस्।

## समीक्षा र आत्म-अध्ययन

[स्ट्यानफोर्डको यो पेपर](https://web.stanford.edu/~jurafsky/slp3/5.pdf) का पहिलो केही पृष्ठहरू पढ्नुहोस् जसमा लजिस्टिक रिग्रेसनका केही व्यावहारिक प्रयोगहरू उल्लेख छन्। ती कार्यहरूको बारेमा सोच्नुहोस् जुन हामीले अहिलेसम्म अध्ययन गरेका रिग्रेसनका प्रकारहरूमध्ये कुनै एकका लागि उपयुक्त छन्। कुन प्रकार सबैभन्दा राम्रो काम गर्ला?



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको छ। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
